In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from plato.planets.metrics import PlanetPopulationMetrics
from plato.planets.populations import PopulationModel
from plato.stars import filter_p1_targets, filter_valid_targets
from plato.visualisation import (
    FigureProcessor,
    contour_plot,
    get_earth_marker,
    get_palette,
    set_plot_defaults,
)

In [3]:
# set_plot_defaults()

figure_directory = f"../figures/06_planet_populations/"
save = True

In [4]:
LOPS2 = pd.read_csv(f"../data/processed/LOPS2_targets.csv")
LOPN1 = pd.read_csv(f"../data/processed/LOPN1_targets.csv")

fields = pd.concat([LOPS2, LOPN1])
fields = filter_valid_targets(fields)
fields = fields[
    [
        "Radius",
        "Mass",
        "[Fe/H]",
        "u1",
        "u2",
        "gaiaV",
        "n_cameras",
        "Population",
    ]
]

fields["cos_i"] = 0
fields["sigma_star"] = 10e-6
fields = fields.rename(
    columns={
        "Radius": "R_star",
        "Mass": "M_star",
        "gaiaV": "Magnitude_V",
    }
)

p1_sample = filter_p1_targets(fields)

In [5]:
model = PopulationModel(
    fields,
    num_embryos=100,
    additional_columns=["Population"],
)

In [6]:
metrics = PlanetPopulationMetrics()
num_mocks = 2

In [7]:
r = []

for _ in range(num_mocks):
    mock = model.create_mock_population(additional_columns=["Population"])
    q = []
    for comp in [None, "Thin Disk", "Thick Disk", "Halo"]:
        number_of_systems = (
            len(fields) if comp is None else len(fields[fields["Population"] == comp])
        )

        m = metrics.calculate_metrics(
            mock, number_of_systems=number_of_systems, population=comp
        ).reindex(
            [
                "Earth",
                "Super-Earth",
                "Neptunian",
                "Sub-Giant",
                "Giant",
            ]
        )
        m["Population"] = "All" if comp is None else comp

        # make population primary index
        m = m.set_index("Population", append=True).reorder_levels(
            ["Population", "Planet Category"]
        )

        q.append(m)

    r.append(pd.concat(q))

t = metrics.calculate_metrics_stats(
    r, grouping_column=["Population", "Planet Category"]
).reindex(["All", "Thin Disk", "Thick Disk", "Halo"], level=0)

In [36]:
t = metrics.calculate_metrics_stats(
    r, grouping_column=["Population", "Planet Category"]
).reindex(
    index=pd.MultiIndex.from_product(
        [
            ["All", "Thin Disk", "Thick Disk", "Halo"],
            ["Earth", "Super-Earth", "Neptunian", "Sub-Giant", "Giant"],
        ],
        names=t.index.names,
    )
)

In [37]:
t

Number of Planets                    \
                                      Median     16th     84th   
Population Planet Category                                       
All        Earth                   1215468.0  1372.92  1372.92   
           Super-Earth              990910.0   797.64   797.64   
           Neptunian                 89782.5    91.46    91.46   
           Sub-Giant                 25991.0    33.32    33.32   
           Giant                     69266.0   162.52   162.52   
Thin Disk  Earth                    792278.5   977.50   977.50   
           Super-Earth              660792.5   804.10   804.10   
           Neptunian                 59980.0     2.04     2.04   
           Sub-Giant                 17052.0    73.44    73.44   
           Giant                     45485.0   110.16   110.16   
Thick Disk Earth                     45111.0    38.76    38.76   
           Super-Earth               23957.5    92.14    92.14   
           Neptunian                  1752.0     6.12     6.12   
           Sub-Giant                   481.5     3.06     3.06   
           Giant                      1289.5    22.10    22.10   
Halo       Earth                      2845.5    28.22    28.22   
           Super-Earth                 551.5     9.86     9.86   
           Neptunian                    18.5     1.70     1.70   
           Sub-Giant                    18.5     1.70     1.70   
           Giant                        24.5     3.74     3.74   

                           System Fraction                      \
                                    Median      16th      84th   
Population Planet Category                                       
All        Earth                  0.887457  0.000030  0.000030   
           Super-Earth            0.743287  0.000196  0.000196   
           Neptunian              0.233601  0.000109  0.000109   
           Sub-Giant              0.074403  0.000116  0.000116   
           Giant                  0.168278  0.000361  0.000361   
Thin Disk  Earth                  0.888948  0.000359  0.000359   
           Super-Earth            0.752776  0.000018  0.000018   
           Neptunian              0.239419  0.000172  0.000172   
           Sub-Giant              0.075137  0.000397  0.000397   
           Giant                  0.168965  0.000401  0.000401   
Thick Disk Earth                  0.891627  0.000039  0.000039   
           Super-Earth            0.639047  0.001380  0.001380   
           Neptunian              0.140960  0.000118  0.000118   
           Sub-Giant              0.042735  0.000276  0.000276   
           Giant                  0.094921  0.001065  0.001065   
Halo       Earth                  0.882294  0.000684  0.000684   
           Super-Earth            0.429577  0.018471  0.018471   
           Neptunian              0.020121  0.001368  0.001368   
           Sub-Giant              0.021127  0.002052  0.002052   
           Giant                  0.035211  0.004789  0.004789   

                           Occurrence Rate                     Multiplicity  \
                                    Median      16th      84th       Median   
Population Planet Category                                                    
All        Earth                  4.604813  0.005201  0.005201     5.188775   
           Super-Earth            3.754073  0.003022  0.003022     5.050637   
           Neptunian              0.340142  0.000346  0.000346     1.456077   
           Sub-Giant              0.098467  0.000126  0.000126     1.323439   
           Giant                  0.262415  0.000616  0.000616     1.559411   
Thin Disk  Earth                  4.600974  0.005677  0.005677     5.175750   
           Super-Earth            3.837399  0.004670  0.004670     5.097665   
           Neptunian              0.348320  0.000012  0.000012     1.454856   
           Sub-Giant              0.099026  0.000426  0.000426     1.317942   
           Giant                  0.26414

In [7]:
results = {}

for num_embryos in [10, 20, 30, 100]:

    print(f"Number of embryos: {num_embryos}, Component: {component}")
    model = PopulationModel(
        fields,
        num_embryos=num_embryos,
        additional_columns=["Population"],
    )

    mock_populations = []
    for i in range(num_mocks):
        mock_populations.append(
            model.create_mock_population(
                add_planet_category=True,
                additional_columns=["Population"],
            )
        )

    del model

    results[num_embryos, component] = metrics.calculate_agg_metrics(
        mock_populations,
        number_of_systems=len(fields),
        population=component,
    )

Number of embryos: 10, Component: Halo
Number of embryos: 10, Component: Thick Disk


: 